In [1]:
%load_ext autoreload
%autoreload 2
from IPython.core import display 
display.display(display.HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import PIL
from IPython import display
from ipycanvas import canvas
import time

In [3]:
from flatland.envs import malfunction_generators as malgen
from flatland.envs.agent_utils import EnvAgent
#from flatland.envs import sparse_rail_gen as spgen
from flatland.envs import rail_generators as rail_gen
from flatland.envs import agent_chains as ac
from flatland.envs.rail_env import RailEnv, RailEnvActions
from flatland.envs.persistence import RailEnvPersister
from flatland.utils.rendertools import RenderTool
from flatland.utils import env_edit_utils as eeu

In [ ]:
nAg=2
bUCF=False
bReverseStart=False
env, envModel = eeu.makeTestEnv(sName="merging_spurs", nAg=nAg, bUCF=bUCF)
oRT = RenderTool(env, show_debug=True)
oRT.render_env(show_rowcols=True,  show_inactive_agents=True, show_observations=False)
oCan = canvas.Canvas(size=(600,300))
oCan.put_image_data(oRT.get_image())
print(f"UCF: {bUCF}")
oCan


UCF: False


Canvas(layout=Layout(height='300px', width='600px'), size=(600, 300))

In [ ]:
iPauseStep=10
iPauseLen = 5

for iStep in range(25):
    
    if bReverseStart:
        iAgentStart = max((nAg - 2 - iStep*2), 0)
    else:
        iAgentStart = 0
    dActions = { i:int(RailEnvActions.MOVE_FORWARD) for i in range(iAgentStart, len(env.agents)) }
    
    if (iStep >= iPauseStep) and (iStep < iPauseStep + iPauseLen):
        dActions[0] = RailEnvActions.STOP_MOVING
        
    #print(dActions)
    
    env.step(dActions)
    
    oRT.render_env(show_rowcols=True,  show_inactive_agents=True, show_observations=False)
    
    aImg = oRT.get_image()
    pilImg = PIL.Image.fromarray(aImg)
    oCan.put_image_data(aImg)
    with open(f"tmp-images/img-{iStep:03d}.png", "wb") as fImg:
        pilImg.save(fImg)
    
    time.sleep(0.5)

In [ ]:
#!ffmpeg -i ./tmp-images/img-%03d.png -vcodec mpeg4 -filter:v "setpts=8.0*PTS" -y movie_nucf_stop.mp4